# Dog Breed Classification Using Transfer Learning

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [3]:
# Install smdebug
!pip install smdebug

In [4]:
# Import packages
import sagemaker
import boto3
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile
from sagemaker.pytorch import PyTorch

import smdebug
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from markupsafe import Markup

from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
import IPython


[2022-04-28 09:45:58.891 ip-172-16-30-72:18410 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


In [4]:
sagemaker.get_execution_role()

'arn:aws:iam::700919767769:role/service-role/AmazonSageMaker-ExecutionRole-20220423T222358'

## Dataset
The dog breed classification dataset provided by Udacity has been used here. The dataset contains images from 133 dog breeds divided into training, testing and validation datasets.

In [ ]:
# Fetch and upload the data to AWS S3

# Command to download and unzip data
#!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
#!unzip dogImages.zip -d dataset/

In [8]:
#!ls /root/'Deep Learning Project'/dataset

dogImages  dogImages.zip


In [10]:
#!aws s3 cp /root/'Deep Learning Project'/dataset s3://image-classification-dog-breed --recursive

upload: ../dataset/dogImages/test/001.Affenpinscher/Affenpinscher_00003.jpg to s3://image-classification-dog-breed/dogImages/test/001.Affenpinscher/Affenpinscher_00003.jpg
upload: ../dataset/dogImages/test/001.Affenpinscher/Affenpinscher_00023.jpg to s3://image-classification-dog-breed/dogImages/test/001.Affenpinscher/Affenpinscher_00023.jpg
upload: ../dataset/dogImages/test/001.Affenpinscher/Affenpinscher_00036.jpg to s3://image-classification-dog-breed/dogImages/test/001.Affenpinscher/Affenpinscher_00036.jpg
upload: ../dataset/dogImages/test/001.Affenpinscher/Affenpinscher_00047.jpg to s3://image-classification-dog-breed/dogImages/test/001.Affenpinscher/Affenpinscher_00047.jpg
upload: ../dataset/dogImages/test/001.Affenpinscher/Affenpinscher_00058.jpg to s3://image-classification-dog-breed/dogImages/test/001.Affenpinscher/Affenpinscher_00058.jpg
upload: ../dataset/dogImages/test/001.Affenpinscher/Affenpinscher_00048.jpg to s3://image-classification-dog-breed/dogImages/test/001.Affenp

## Hyperparameter Tuning

In this part a resnet18 pretrained model has been used to perform hyperparameter tuning - such that it the best hyperparameters are determined for the predictor model. 
The hyperparameters being tuned are:

1) Learning rate    --> Continuous parameter with values tuned between 0.001 and 0,1

2) Batch size       --> Categorical parameter with specific list of trial values

3) Number of epochs --> Integer parameter between 1 and 2(inclusive)

The training and testing script in `hpo.py` is used to perform hyperparameter tuning.

In [6]:
# Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512]),
    "epochs": IntegerParameter(1, 2)
}

In [ ]:
# The objective metric to be used by the Hyperparameter Tuning jobs is the Test Accuracy of the model on the validation dataset

objective_metric_name = "Accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": "Accuracy", "Regex": "Test set: Accuracy: ([0-9\\.]+)%"}]


Configuring the estimated to run 6 total hyperparameter tuner jobs with a allowed parallel job count of 2

In [8]:
# Create estimators for your HPs
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="hpo.py",
    role=sagemaker.get_execution_role(),
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.2xlarge"
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=6,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [ ]:

inputs_train = 's3://image-classification-dog-breed/dogImages/train/'
inputs_test = 's3://image-classification-dog-breed/dogImages/test/'
# Passing input data channels to the tuner --> the data from the input channels will be downloaded on the container image in the /opt/ml/input directory
tuner.fit({"training": inputs_train,"test":inputs_test})


.....................................................................................................................................................................................................................................................

In [11]:
# Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2022-04-27 21:05:25 Starting - Preparing the instances for training
2022-04-27 21:05:25 Downloading - Downloading input data
2022-04-27 21:05:25 Training - Training image download completed. Training in progress.
2022-04-27 21:05:25 Uploading - Uploading generated training model
2022-04-27 21:05:25 Completed - Training job completed


{'_tuning_objective_metric': '"Accuracy"',
 'batch-size': '"128"',
 'epochs': '2',
 'lr': '0.036756477351601646',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2022-04-27-20-31-24-050"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-700919767769/pytorch-training-2022-04-27-20-31-24-050/source/sourcedir.tar.gz"'}

The optimum hyperparameters being determined by the tuning job are:

1) Learning rate    --> 0.036756477351601646

2) Batch size       --> 128

3) Number of epochs --> 2
